In [39]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageEnhance
import numpy as np
import matplotlib.pyplot as plt
from skimage import color, filters, segmentation, morphology, img_as_ubyte
from sklearn.mixture import GaussianMixture
from scipy import ndimage as ndi
from skimage.segmentation import random_walker
import torch
import torchvision.transforms as T
import torchvision.models.segmentation as models
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import cv2


In [40]:
# ==============================
# Chapter 1 - Các hàm xử lý cơ bản
# ==============================

def to_gray(img):
    return Image.fromarray(img_as_ubyte(color.rgb2gray(np.array(img))))

def to_hsv(img):
    hsv = color.rgb2hsv(np.array(img))
    return Image.fromarray(img_as_ubyte(hsv[:, :, 0]))  # hiển thị kênh Hue

def rotate_90(img):
    return img.rotate(90, expand=True)

def flip_horizontal(img):
    return img.transpose(Image.FLIP_LEFT_RIGHT)

def crop_center(img):
    w, h = img.size
    left, top = w//4, h//4
    right, bottom = 3*w//4, 3*h//4
    return img.crop((left, top, right, bottom))

def adjust_brightness(img, factor=1.5):
    enhancer = ImageEnhance.Brightness(img)
    return enhancer.enhance(factor)

def show_histogram(img):
    arr = np.array(img.convert("L"))
    fig, ax = plt.subplots(figsize=(4,3))
    ax.hist(arr.flatten(), bins=256, color="gray", alpha=0.7)
    ax.set_title("Histogram Ảnh")
    ax.set_xlabel("Giá trị pixel")
    ax.set_ylabel("Tần suất")

    top = tk.Toplevel()
    top.title("Histogram")
    canvas = FigureCanvasTkAgg(fig, master=top)
    canvas.draw()
    canvas.get_tk_widget().pack(fill="both", expand=True)
    return img

def rgb_to_lab_gray(img):
    """
    Chuyển RGB sang không gian màu Lab, 
    đặt kênh a và b = 0 để chỉ giữ lại độ sáng (kênh L).
    Kết quả: ảnh xám tự nhiên hơn.
    """
    arr = np.array(img).astype(np.float32) / 255.0
    lab = color.rgb2lab(arr)
    lab[...,1] = 0
    lab[...,2] = 0
    rgb_back = color.lab2rgb(lab)
    return Image.fromarray(img_as_ubyte(rgb_back))

def lab_adjust_brightness(img, delta_L=30):
    """
    Thay đổi độ sáng bằng cách tăng/giảm giá trị kênh L trong Lab.
    delta_L > 0 => ảnh sáng hơn, delta_L < 0 => ảnh tối hơn.
    """
    arr = np.array(img).astype(np.float32) / 255.0
    lab = color.rgb2lab(arr)
    lab[...,0] = np.clip(lab[...,0] + delta_L, 0, 100)
    rgb_back = color.lab2rgb(lab)
    return Image.fromarray(img_as_ubyte(rgb_back))

def affine_transform_demo(img):
    """
    Biểu diễn các phép biến đổi affine:
      - Scale (co giãn ảnh)
      - Rotate (xoay ảnh)
      - Shear (biến dạng xiên)
    Trả về ảnh ghép để so sánh.
    """
    arr = np.array(img.convert("L")).astype(np.float32) / 255.0
    h, w = arr.shape

    # Scale
    tform_scale = AffineTransform(scale=(0.75, 1.25))
    scaled = warp(arr, tform_scale.inverse, output_shape=(h, w))

    # Rotate 30 độ
    rotated = sk_rotate(arr, angle=30, resize=False)

    # Shear
    tform_shear = AffineTransform(shear=0.4)
    sheared = warp(arr, tform_shear.inverse, output_shape=(h, w))

    # Chuyển sang ảnh PIL để hiển thị
    orig = Image.fromarray(img_as_ubyte(arr))
    scaled_pil = Image.fromarray(img_as_ubyte(scaled))
    rotated_pil = Image.fromarray(img_as_ubyte(rotated))
    sheared_pil = Image.fromarray(img_as_ubyte(sheared))

    return concat_horiz([orig, scaled_pil, rotated_pil, sheared_pil])

def perspective_transform_demo(img):
    """
    Biến đổi phối cảnh (homography):
    - Chọn 4 điểm ở ảnh nguồn (quad)
    - Ánh xạ về hình chữ nhật chuẩn
    """
    arr = np.array(img.convert("RGB"))
    h, w = arr.shape[:2]

    # Tạo 4 điểm nguồn (giả sử lấy gần góc)
    src = np.array([[0.1*w,0.1*h],[0.9*w,0.15*h],[0.85*w,0.9*h],[0.15*w,0.85*h]], dtype=np.float32)
    dst = np.array([[0,0],[w-1,0],[w-1,h-1],[0,h-1]], dtype=np.float32)

    H = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(arr, H, (w, h))

    return Image.fromarray(warped)

def sketch_dodge(img, sigma=0.5, k=1.6, tau=0.98, phi=200):
    """
    Sketch bằng Difference of Gaussian (DOG).
    """
    gray = np.array(img.convert("L"), dtype=np.float32) / 255.0

    # Gaussian blur
    g1 = ndi.gaussian_filter(gray, sigma)
    g2 = ndi.gaussian_filter(gray, sigma * k)

    # DOG công thức trong paper
    dog = g1 - tau * g2

    # Nhấn mạnh biên bằng tanh
    dog = np.tanh(phi * dog)

    # Chuẩn hóa
    dog = (dog - dog.min()) / (dog.max() - dog.min())
    dog = (dog * 255).astype(np.uint8)

    return Image.fromarray(dog)


def sketch_xdog(img, sigma=0.5, k=1.6, tau=0.98, epsilon=0.01, phi=200):
    """
    Sketch bằng Extended Difference of Gaussian (XDOG).
    """
    gray = np.array(img.convert("L"), dtype=np.float32) / 255.0

    # Gaussian blur
    g1 = ndi.gaussian_filter(gray, sigma)
    g2 = ndi.gaussian_filter(gray, sigma * k)

    # DOG
    dog = g1 - tau * g2

    # Áp dụng công thức XDOG
    xdog = np.where(dog >= epsilon, 1.0, 1.0 + np.tanh(phi * (dog - epsilon)))

    # Chuẩn hóa
    xdog = (xdog - xdog.min()) / (xdog.max() - xdog.min())
    xdog = (xdog * 255).astype(np.uint8)

    return Image.fromarray(xdog)

def cartoon_effect(img):
    """
    Cartoon effect:
      1. Giảm nhiễu bằng bilateral filter (lặp nhiều lần)
      2. Phát hiện biên bằng adaptive threshold
      3. Kết hợp lại để tạo ảnh phong cách hoạt hình
    """
    arr = np.array(img.convert("RGB"))
    bgr = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)

    # Bilateral filtering trên pyramid
    tmp = bgr.copy()
    for _ in range(2):
        tmp = cv2.pyrDown(tmp)
    for _ in range(7):
        tmp = cv2.bilateralFilter(tmp, d=9, sigmaColor=75, sigmaSpace=75)
    for _ in range(2):
        tmp = cv2.pyrUp(tmp)
    tmp = cv2.resize(tmp, (bgr.shape[1], bgr.shape[0]))

    # Edge mask
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(gray, 7)
    edges = cv2.adaptiveThreshold(blur, 255,
                                  cv2.ADAPTIVE_THRESH_MEAN_C,
                                  cv2.THRESH_BINARY, 9, 2)
    edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    cartoon = cv2.bitwise_and(tmp, edges)
    cartoon = cv2.cvtColor(cartoon, cv2.COLOR_BGR2RGB)
    return Image.fromarray(cartoon)

# ========== tiện ích hỗ trợ ==========
from skimage.transform import AffineTransform, warp, rotate as sk_rotate
def concat_horiz(images):
    """Ghép ảnh ngang để so sánh"""
    widths = [im.width for im in images]
    heights = [im.height for im in images]
    total_w = sum(widths)
    max_h = max(heights)
    new = Image.new("RGB",(total_w,max_h),(255,255,255))
    x=0
    for im in images:
        new.paste(im,(x,(max_h-im.height)//2))
        x+=im.width
    return new

In [41]:
# ==============================
# Chapter 6 : Phân đoạn ảnh
# ==============================

def otsu_threshold(img):
    gray = color.rgb2gray(np.array(img))
    t = filters.threshold_otsu(gray)
    binary = gray > t
    return Image.fromarray(img_as_ubyte(binary))

def riddler_calvard_threshold(img):
    gray = color.rgb2gray(np.array(img))
    t = gray.mean()
    for _ in range(50):
        g1, g2 = gray[gray <= t], gray[gray > t]
        if len(g1) == 0 or len(g2) == 0:
            break
        t_new = 0.5 * (g1.mean() + g2.mean())
        if abs(t_new - t) < 1e-3:
            break
        t = t_new
    binary = gray > t
    return Image.fromarray(img_as_ubyte(binary))

def watershed_segmentation(img):
    gray = color.rgb2gray(np.array(img))
    thresh = filters.threshold_otsu(gray)
    bw = gray > thresh
    distance = ndi.distance_transform_edt(bw)
    local_maxi = morphology.h_maxima(distance, 10)
    markers, _ = ndi.label(local_maxi)
    labels = segmentation.watershed(-distance, markers, mask=bw)
    return Image.fromarray(img_as_ubyte(color.label2rgb(labels, bg_label=0)))

def skin_segmentation(img):
    """Phân đoạn da bằng GMM trong không gian YCbCr."""
    ycbcr = color.rgb2ycbcr(np.array(img) / 255.0)
    feats = ycbcr[..., 1:3].reshape(-1, 2)  # chỉ lấy Cb, Cr
    gmm = GaussianMixture(n_components=2, random_state=0).fit(feats)
    labels = gmm.predict(feats).reshape(ycbcr.shape[:2])
    # chọn cluster có số pixel nhiều nhất làm "da"
    skin_comp = np.bincount(labels.flatten()).argmax()
    mask = labels == skin_comp
    result = np.array(img).copy()
    result[~mask] = 0
    return Image.fromarray(result)


def som_segmentation(img, n_clusters=4):
    """Phân đoạn ảnh bằng K-means (thay thế SOM)."""
    from sklearn.cluster import KMeans
    arr = np.array(img)
    h, w, c = arr.shape
    arr2 = arr.reshape(-1, c)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(arr2)
    labels = kmeans.labels_.reshape(h, w)
    seg = color.label2rgb(labels, arr, kind='avg')
    return Image.fromarray(img_as_ubyte(seg))


def random_walk_segmentation(img):
    gray = color.rgb2gray(np.array(img))
    markers = np.zeros(gray.shape, dtype=np.int32)
    markers[gray < 0.3] = 1
    markers[gray > 0.7] = 2
    labels = random_walker(gray, markers, beta=10, mode='cg')  # cg nhanh hơn
    seg = color.label2rgb(labels, np.array(img), kind='avg')
    return Image.fromarray(img_as_ubyte(seg))

def mri_gmm_em(img, n_components=4):
    """Phân đoạn ảnh MRI T1 bằng GMM-EM."""
    gray = np.array(img.convert("L"), dtype=np.float32) / 255.0  # chuẩn hóa 0-1
    X = gray.reshape(-1, 1)

    gmm = GaussianMixture(n_components=n_components,
                          covariance_type="tied",
                          random_state=0).fit(X)
    labels = gmm.predict(X).reshape(gray.shape)

    # sắp xếp lại label theo mean intensity
    means = gmm.means_.flatten()
    order = np.argsort(means)
    mapping = {old: new for new, old in enumerate(order)}
    labels = np.vectorize(mapping.get)(labels)

    # Tạo ảnh màu từ labels (không dùng gray làm nền nữa)
    seg = color.label2rgb(labels, colors=['brown','orange','pink','blue'], bg_label=0)
    return Image.fromarray(img_as_ubyte(seg))



def mri_fcn(img):
    """Dùng FCN pretrained trong torchvision để phân đoạn ảnh tổng quát."""
    model = models.fcn_resnet50(weights="DEFAULT").eval()
    preprocess = T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(img).unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)['out'][0]
    pred = output.argmax(0).byte().cpu().numpy()
    seg = color.label2rgb(pred, np.array(img), kind="avg")
    return Image.fromarray(img_as_ubyte(seg))



In [42]:
# ==============================
# GUI 
# ==============================
class ImageDemoApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Processing Demos (Ch.1 & Ch.6)")
        self.root.geometry("1100x700")

        self.original_img = None   # Ảnh gốc
        self.current_img = None    # Ảnh hiện tại
        self.display_img = None

        # ===== Sidebar có thanh cuộn =====
        sidebar = tk.Frame(root, width=280, bg="#f0f0f0")
        sidebar.pack(side="left", fill="y")

        canvas = tk.Canvas(sidebar, bg="#f0f0f0", highlightthickness=0)
        scrollbar = tk.Scrollbar(sidebar, orient="vertical", command=canvas.yview)
        self.scrollable_frame = tk.Frame(canvas, bg="#f0f0f0")

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )

        canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)

        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        # ===== Các nút bấm =====
        tk.Button(self.scrollable_frame, text="📂 Tải ảnh lên", command=self.upload_image,
                  bg="#4CAF50", fg="white", font=("Arial", 12), width=22).pack(pady=10)

        tk.Button(self.scrollable_frame, text="↩️ Hoàn tác (Undo)", command=self.undo,
                  bg="#FF9800", fg="white", font=("Arial", 12), width=22).pack(pady=5)

        # --- Chương 1 cơ bản ---
        tk.Label(self.scrollable_frame, text="Chương 1 - Xử lý cơ bản", 
                 font=("Arial", 12, "bold"), bg="#f0f0f0").pack(pady=5)
        tk.Button(self.scrollable_frame, text="Ảnh Xám", command=lambda: self.apply_demo(to_gray)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Chuyển HSV (Hue)", command=lambda: self.apply_demo(to_hsv)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Xoay 90°", command=lambda: self.apply_demo(rotate_90)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Lật Ngang", command=lambda: self.apply_demo(flip_horizontal)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Cắt Giữa", command=lambda: self.apply_demo(crop_center)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Tăng Độ Sáng", command=lambda: self.apply_demo(adjust_brightness)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Xem Histogram", command=lambda: self.apply_demo(show_histogram)).pack(pady=3)

        # --- Chương 1 nâng cao ---
        tk.Label(self.scrollable_frame, text="Chương 1 - Demo nâng cao", 
                 font=("Arial", 12, "bold"), bg="#f0f0f0").pack(pady=5)
        tk.Button(self.scrollable_frame, text="RGB → Lab Gray", command=lambda: self.apply_demo(rgb_to_lab_gray)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Lab Brightness +30", command=lambda: self.apply_demo(lambda im: lab_adjust_brightness(im,30))).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Affine Transform", command=lambda: self.apply_demo(affine_transform_demo)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Perspective Transform", command=lambda: self.apply_demo(perspective_transform_demo)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Sketch (Dodge)", command=lambda: self.apply_demo(sketch_dodge)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Sketch (XDOG)", command=lambda: self.apply_demo(sketch_xdog)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Cartoon Effect", command=lambda: self.apply_demo(cartoon_effect)).pack(pady=3)

        # --- Chương 6 ---
        tk.Label(self.scrollable_frame, text="Chương 6 - Phân đoạn ảnh", 
                 font=("Arial", 12, "bold"), bg="#f0f0f0").pack(pady=10)
        tk.Button(self.scrollable_frame, text="Ngưỡng Otsu", command=lambda: self.apply_demo(otsu_threshold)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Ngưỡng Riddler-Calvard", command=lambda: self.apply_demo(riddler_calvard_threshold)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Phân đoạn Watershed", command=lambda: self.apply_demo(watershed_segmentation)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Phân đoạn Da (GMM-EM)", command=lambda: self.apply_demo(skin_segmentation)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Phân đoạn SOM", command=lambda: self.apply_demo(som_segmentation)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="Phân đoạn Random Walk", command=lambda: self.apply_demo(random_walk_segmentation)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="MRI Segmentation (GMM-EM)", command=lambda: self.apply_demo(mri_gmm_em)).pack(pady=3)
        tk.Button(self.scrollable_frame, text="MRI Segmentation (FCN)", command=lambda: self.apply_demo(mri_fcn)).pack(pady=3)

        # ===== Vùng hiển thị ảnh =====
        self.img_label = tk.Label(root, bg="black")
        self.img_label.pack(side="right", expand=True, fill="both")

    # ===== Các hàm xử lý ảnh =====
    def upload_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png *.jpeg *.bmp")])
        if not file_path:
            return
        self.original_img = Image.open(file_path).convert("RGB")
        self.current_img = self.original_img.copy()
        self.display(self.current_img)

    def apply_demo(self, func):
        if self.current_img is None:
            messagebox.showwarning("Chưa có ảnh", "Vui lòng tải ảnh lên trước!")
            return
        try:
            result = func(self.current_img)
            if result is not None:
                self.current_img = result
                self.display(self.current_img)
        except Exception as e:
            messagebox.showerror("Lỗi", str(e))

    def undo(self):
        if self.original_img is None:
            messagebox.showinfo("Hoàn tác", "Chưa có ảnh để hoàn tác.")
            return
        self.current_img = self.original_img.copy()
        self.display(self.current_img)

    def display(self, img):
        w, h = img.size
        max_w, max_h = 750, 650
        scale = min(max_w / w, max_h / h, 1.0)
        img_resized = img.resize((int(w * scale), int(h * scale)))

        self.display_img = ImageTk.PhotoImage(img_resized)

        # Fix giữ tham chiếu để không bị TclError
        self.img_label.config(image=self.display_img)
        self.img_label.image = self.display_img


In [43]:
if __name__ == "__main__": 
    root = tk.Tk() 
    app = ImageDemoApp(root) 
    root.mainloop() 